# Understanding retries
Here description of what this does.

In [1]:
import datetime
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import scan
import re

import numpy as np
import pandas as pd

es = Elasticsearch([{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)
jobs = []

corepower_agis = {}

corepower_agis['AGLT2_LMEM']                         = 10.3088172502
corepower_agis['AGLT2_MCORE']                        = 10.3088172502
corepower_agis['AGLT2_SL6']                          = 10.3088172502
corepower_agis['AGLT2_TEST']                         = 10.3088172502
corepower_agis['AM-04-YERPHI']                       = 10.00001
corepower_agis['ANALY_AGLT2_SL6']                    = 10.3088172502
corepower_agis['ANALY_AGLT2_TEST_SL6-condor']        = 10.3088172502
corepower_agis['ANALY_AGLT2_TIER3_TEST']             = 10.3088172502
corepower_agis['ANALY_AM-04-YERPHI']                 = 10.00001
corepower_agis['ANALY_ANLASC']                       = 10.00001
corepower_agis['ANALY_ANLASC_Argo']                  = 10.00001
corepower_agis['ANALY_ANLASC_T3Test']                = 10.00001
corepower_agis['ANALY_Anselm_HPC']                   = 9.98157248157
corepower_agis['ANALY_Arizona']                      = 10.0
corepower_agis['ANALY_ARNES']                        = 10.00001
corepower_agis['ANALY_ARNES_DIRECT']                 = 10.00001
corepower_agis['ANALY_AUSTRALIA']                    = 11.375
corepower_agis['ANALY_AUSTRALIA_TEST']               = 11.375
corepower_agis['ANALY_BEIJING']                      = 18.5810810811
corepower_agis['ANALY_BHAM_SL6']                     = 14.1622474747
corepower_agis['ANALY_BNL_CLOUD']                    = 9.98157248157
corepower_agis['ANALY_BNL_EC2E1']                    = 9.98157248157
corepower_agis['ANALY_BNL_EC2W1']                    = 9.98157248157
corepower_agis['ANALY_BNL_EC2W2']                    = 9.98157248157
corepower_agis['ANALY_BNL_LOCAL']                    = 9.98157248157
corepower_agis['ANALY_BNL_LONG']                     = 9.98157248157
corepower_agis['ANALY_BNL_MCORE']                    = 9.98157248157
corepower_agis['ANALY_BNL_SE_Test']                  = 9.98157248157
corepower_agis['ANALY_BNL_SHORT']                    = 9.98157248157
corepower_agis['ANALY_BNL_T3']                       = 9.98157248157
corepower_agis['ANALY_BNL_test2']                    = 9.98157248157
corepower_agis['ANALY_BNL_Test_2_CE_1']              = 9.98157248157
corepower_agis['ANALY_BNL_test3']                    = 9.98157248157
corepower_agis['ANALY_BNL_test']                     = 9.98157248157
corepower_agis['ANALY_BU_ATLAS_Tier2_SL6']           = 20.0126760563
corepower_agis['ANALY_CAM_SL6']                      = 14.5588235294
corepower_agis['ANALY_CERN_CLOUD']                   = 9.72511065833
corepower_agis['ANALY_CERN_GLEXECDEV']               = 9.72511065833
corepower_agis['ANALY_CERN_SHORT']                   = 9.72511065833
corepower_agis['ANALY_CERN_SLC6']                    = 9.72511065833
corepower_agis['ANALY_CERN_TEST']                    = 9.72511065833
corepower_agis['ANALY_CERNVM_test']                  = 9.72511065833
corepower_agis['ANALY_CONNECT']                      = 10.2945472249
corepower_agis['ANALY_CONNECT_SHORT']                = 10.2945472249
corepower_agis['ANALY_CPPM']                         = 10.879979571
corepower_agis['ANALY_CSCS']                         = 11.5520238713
corepower_agis['ANALY_CSCS-HPC']                     = 11.5520238713
corepower_agis['ANALY_CYF']                          = 14.1999671916
corepower_agis['ANALY_DCSC']                         = 12.0302510131
corepower_agis['ANALY_DESY-HH']                      = 11.2781627838
corepower_agis['ANALY_DESY-HH_TEST']                 = 11.2781627838
corepower_agis['ANALY_DESY-ZN']                      = 16.1600506757
corepower_agis['ANALY_DESY-ZN_XRD']                  = 16.1600506757
corepower_agis['ANALY_DRESDEN']                      = 10.00001
corepower_agis['ANALY_ECDF_SL6']                     = 12.8996212121
corepower_agis['ANALY_ECDF_SL7']                     = 12.8996212121
corepower_agis['ANALY_FMPhI-UNIBA']                  = 10.5133928571
corepower_agis['ANALY_FREIBURG']                     = 8.42007926024
corepower_agis['ANALY_FZK']                          = 13.0793336979
corepower_agis['ANALY_FZK_HI']                       = 13.0793336979
corepower_agis['ANALY_FZK_SHORT']                    = 13.0793336979
corepower_agis['ANALY_FZU']                          = 8.62183235867
corepower_agis['ANALY_GLASGOW_SL6']                  = 8.740063593
corepower_agis['ANALY_GOEGRID']                      = 10.2658730159
corepower_agis['ANALY_GR-01-AUTH']                   = 10.00001
corepower_agis['ANALY_GRIF-IRFU']                    = 10.7483226968
corepower_agis['ANALY_GRIF-LAL_HTCondor']            = 10.7483226968
corepower_agis['ANALY_GRIF-LPNHE']                   = 10.7483226968
corepower_agis['ANALY_HEPHY-UIBK']                   = 8.45769230769
corepower_agis['ANALY_HPC2N']                        = 12.0302510131
corepower_agis['ANALY_HU_ATLAS_Tier2']               = 12.5894097222
corepower_agis['ANALY_IAAS']                         = 14.3604166667
corepower_agis['ANALY_IAAS_TEST']                    = 14.3604166667
corepower_agis['ANALY_IEPSAS-Kosice']                = 15.0191780822
corepower_agis['ANALY_IFAE']                         = 12.1205357143
corepower_agis['ANALY_IFIC']                         = 12.9677419355
corepower_agis['ANALY_IHEP']                         = 8.79993297587
corepower_agis['ANALY_IL-TAU-HEP-CREAM']             = 11.7509363296
corepower_agis['ANALY_IN2P3-CC']                     = 0.0
corepower_agis['ANALY_INFN-BOLOGNA-T3']              = 10.00001
corepower_agis['ANALY_INFN-COSENZA']                 = 10.00001
corepower_agis['ANALY_INFN-COSENZA-RECAS']           = 8.503
corepower_agis['ANALY_INFN-FRASCATI']                = 9.21590909091
corepower_agis['ANALY_INFN-FRASCATI_PODtest']        = 9.21590909091
corepower_agis['ANALY_INFN-GENOVA']                  = 10.00001
corepower_agis['ANALY_INFN-LECCE']                   = 10.00001
corepower_agis['ANALY_INFN-MILANO-ATLASC']           = 8.20586592179
corepower_agis['ANALY_INFN-NAPOLI']                  = 8.50280269058
corepower_agis['ANALY_INFN-NAPOLI-RECAS']            = 8.50280269058
corepower_agis['ANALY_INFN-ROMA1']                   = 9.33066239316
corepower_agis['ANALY_INFN-ROMA2']                   = 10.00001
corepower_agis['ANALY_INFN-ROMA3']                   = 10.00001
corepower_agis['ANALY_INFN-T1']                      = 10.91
corepower_agis['ANALY_INFN-T1-VWN2_TEST']            = 10.91
corepower_agis['ANALY_INFN-T1-VWN_TEST']             = 10.91
corepower_agis['ANALY_JINR']                         = 12.6984042553
corepower_agis['ANALY_LANCS_SL6']                    = 14.4
corepower_agis['ANALY_LAPP']                         = 11.2559633028
corepower_agis['ANALY_LAPP_TEST']                    = 11.2559633028
corepower_agis['ANALY_LIV_SL6']                      = 10.1992481203
corepower_agis['ANALY_LONG_BNL_LOCAL']               = 9.98157248157
corepower_agis['ANALY_LPC']                          = 12.1723000824
corepower_agis['ANALY_LPSC']                         = 12.0
corepower_agis['ANALY_LRZ']                          = 9.69296461976
corepower_agis['ANALY_LRZ_SL6']                      = 9.69296461976
corepower_agis['ANALY_LRZ_TEST']                     = 9.69296461976
corepower_agis['ANALY_LUNARC']                       = 11.2483237548
corepower_agis['ANALY_MANC_SL6']                     = 9.85640681004
corepower_agis['ANALY_MANC_SL6_SHORT']               = 9.85640681004
corepower_agis['ANALY_MANC_TEST']                    = 9.85640681004
corepower_agis['ANALY_MCGILL']                       = 14.5
corepower_agis['ANALY_MCGILL_DOCKER']                = 14.5
corepower_agis['ANALY_MCGILL_TEST']                  = 14.5
corepower_agis['ANALY_MPPMU']                        = 11.3697916667
corepower_agis['ANALY_MWT2_HIMEM']                   = 10.2945472249
corepower_agis['ANALY_MWT2_HIMEM_MCORE']             = 10.2945472249
corepower_agis['ANALY_MWT2_MCORE']                   = 10.2945472249
corepower_agis['ANALY_MWT2_SL6']                     = 10.2945472249
corepower_agis['ANALY_NCG-INGRID-PT_SL6']            = 8.01984126984
corepower_agis['ANALY_NERSC']                        = 9.98157248157
corepower_agis['ANALY_NIKHEF-ELPROD']                = 13.9899445215
corepower_agis['ANALY_NIKHEF-ELPROD_SHORT']          = 13.9899445215
corepower_agis['ANALY_NSC']                          = 12.0302510131
corepower_agis['ANALY_ORNL_Titan']                   = 9.98157248157
corepower_agis['ANALY_OU_OCHEP_SWT2']                = 12.921875
corepower_agis['ANALY_OX_SL6']                       = 10.1547987616
corepower_agis['ANALY_OX_TEST']                      = 10.1547987616
corepower_agis['ANALY_PIC_SL6']                      = 12.1286846276
corepower_agis['ANALY_PSNC']                         = 51.9035369775
corepower_agis['ANALY_QMUL_HIMEM_SL6']               = 11.2843283582
corepower_agis['ANALY_QMUL_SL6']                     = 11.2843283582
corepower_agis['ANALY_RAL_ECHO']                     = 10.0
corepower_agis['ANALY_RAL_MCORE']                    = 10.0
corepower_agis['ANALY_RALPP_SL6']                    = 10.0
corepower_agis['ANALY_RAL_SL6']                      = 10.0
corepower_agis['ANALY_RHUL_SL6']                     = 10.0268229167
corepower_agis['ANALY_ROMANIA02']                    = 9.6875
corepower_agis['ANALY_ROMANIA07']                    = 10.5637903653
corepower_agis['ANALY_RRC-KI']                       = 11.9201520913
corepower_agis['ANALY_RRC-KI-HPC']                   = 15.5
corepower_agis['ANALY_RRC-KI-T1']                    = 15.5
corepower_agis['ANALY_SARA']                         = 17.2100446429
corepower_agis['ANALY_SARA_bignode']                 = 17.2100446429
corepower_agis['ANALY_SARA_hpc_cloud']               = 17.2100446429
corepower_agis['ANALY_SARA_hpc_cloud_rfio']          = 17.2100446429
corepower_agis['ANALY_SARA_hpc_cloud_xrootd']        = 17.2100446429
corepower_agis['ANALY_SCINET']                       = 12.5
corepower_agis['ANALY_SCINET_TEST']                  = 12.5
corepower_agis['ANALY_SFU']                          = 14.5
corepower_agis['ANALY_SFU_TEST']                     = 14.5
corepower_agis['ANALY_SHEF_SL6']                     = 13.2
corepower_agis['ANALY_SiGNET']                       = 9.79996886675
corepower_agis['ANALY_SiGNET_DIRECT']                = 9.79996886675
corepower_agis['ANALY_SLAC']                         = 12.4958823529
corepower_agis['ANALY_SLAC_LMEM']                    = 12.4958823529
corepower_agis['ANALY_SLAC_SHORT_1HR']               = 12.4958823529
corepower_agis['ANALY_SUSX_SL6']                     = 10.4479166667
corepower_agis['ANALY_SWT2_CPB']                     = 9.65842245989
corepower_agis['ANALY_T1_DE_KIT']                    = 13.0793336979
corepower_agis['ANALY_T1_ES_PIC']                    = 12.1286846276
corepower_agis['ANALY_T1_FR_CCIN2P3']                = 10.2642484677
corepower_agis['ANALY_T1_IT_CNAF']                   = 10.91
corepower_agis['ANALY_T1_RU_JINR']                   = 15.02
corepower_agis['ANALY_T1_UK_RAL']                    = 10.0
corepower_agis['ANALY_T1_UK_RAL_ARC']                = 10.0
corepower_agis['ANALY_T1_US_FNAL']                   = 9.72511065833
corepower_agis['ANALY_T2_AT_Vienna']                 = 11.1
corepower_agis['ANALY_T2_BE_IIHE']                   = 7.64801762115
corepower_agis['ANALY_T2_BE_UCL']                    = 9.2
corepower_agis['ANALY_T2_BR_SPRACE']                 = 11.890625
corepower_agis['ANALY_T2_BR_UERJ']                   = 10.00001
corepower_agis['ANALY_T2_CH_CERN']                   = 9.72511065833
corepower_agis['ANALY_T2_CH_CSCS']                   = 9.72511065833
corepower_agis['ANALY_T2_CN_Beijing']                = 18.5810810811
corepower_agis['ANALY_T2_DE_DESY']                   = 9.72511065833
corepower_agis['ANALY_T2_DE_RWTH']                   = 9.72511065833
corepower_agis['ANALY_T2_EE_Estonia']                = 9.72511065833
corepower_agis['ANALY_T2_ES_CIEMAT']                 = 9.72511065833
corepower_agis['ANALY_T2_ES_IFCA']                   = 11.4299496644
corepower_agis['ANALY_T2_FI_HIP']                    = 12.4741847826
corepower_agis['ANALY_T2_FR_GRIF_IRFU']              = 9.72511065833
corepower_agis['ANALY_T2_FR_GRIF_LLR']               = 9.72511065833
corepower_agis['ANALY_T2_FR_IPHC']                   = 12.567839196
corepower_agis['ANALY_T2_GR_Ioannina']               = 12.0
corepower_agis['ANALY_T2_HU_Budapest']               = 11.719895288
corepower_agis['ANALY_T2_IN_TIFR']                   = 10.0
corepower_agis['ANALY_T2_IT_Bari']                   = 9.72511065833
corepower_agis['ANALY_T2_IT_Legnaro']                = 9.72511065833
corepower_agis['ANALY_T2_IT_Pisa']                   = 9.72511065833
corepower_agis['ANALY_T2_IT_Rome']                   = 9.72511065833
corepower_agis['ANALY_T2_KR_KNU']                    = 9.72511065833
corepower_agis['ANALY_T2_MY_UPM_BIRUNI']             = 10.00001
corepower_agis['ANALY_T2_PK_NCP']                    = 12.1469465649
corepower_agis['ANALY_T2_PL_Warsaw']                 = 16.3999208861
corepower_agis['ANALY_T2_PT_NCG_Lisbon']             = 8.01984126984
corepower_agis['ANALY_T2_RU_IHEP']                   = 8.79993297587
corepower_agis['ANALY_T2_RU_INR']                    = 13.5523255814
corepower_agis['ANALY_T2_RU_ITEP']                   = 13.4051724138
corepower_agis['ANALY_T2_RU_JINR']                   = 12.6984042553
corepower_agis['ANALY_T2_RU_PNPI']                   = 11.0
corepower_agis['ANALY_T2_RU_RRC_KI']                 = 11.9201520913
corepower_agis['ANALY_T2_RU_SINP']                   = 6.0
corepower_agis['ANALY_T2_TH_CUNSTDA']                = 10.00001
corepower_agis['ANALY_T2_TR_METU']                   = 5.19047619048
corepower_agis['ANALY_T2_TW_Taiwan']                 = 10.1397058824
corepower_agis['ANALY_T2_UK_London_Brunel']          = 11.95
corepower_agis['ANALY_T2_UK_London_IC']              = 9.72511065833
corepower_agis['ANALY_T2_UK_SGrid_Bristol']          = 11.1695685087
corepower_agis['ANALY_T2_UK_SGrid_Bristol_long']     = 11.1695685087
corepower_agis['ANALY_T2_UK_SGrid_RALPP']            = 9.72511065833
corepower_agis['ANALY_T2_US_Caltech']                = 9.72511065833
corepower_agis['ANALY_T2_US_Florida']                = 16.5094339623
corepower_agis['ANALY_T2_US_MIT']                    = 9.72511065833
corepower_agis['ANALY_T2_US_Nebraska']               = 9.72511065833
corepower_agis['ANALY_T2_US_UCSD']                   = 9.72511065833
corepower_agis['ANALY_T2_US_Vanderbilt']             = 10.00001
corepower_agis['ANALY_T2_US_Wisconsin']              = 1.97072072072
corepower_agis['ANALY_TAIWAN_PNFS_SL6']              = 10.1397058824
corepower_agis['ANALY_TAIWAN_SL6']                   = 10.1397058824
corepower_agis['ANALY_TAIWAN_TEST']                  = 10.1397058824
corepower_agis['ANALY_TAIWAN_XROOTD_SL6']            = 10.1397058824
corepower_agis['ANALY_TECHNION-HEP-CREAM']           = 11.75
corepower_agis['ANALY_TEST-APF2']                    = 9.98157248157
corepower_agis['ANALY_TEST-APF']                     = 9.98157248157
corepower_agis['ANALY_TOKYO']                        = 18.1097005208
corepower_agis['ANALY_TOKYO_ARC']                    = 18.1097005208
corepower_agis['ANALY_TR-10-ULAKBIM']                = 5.19097222222
corepower_agis['ANALY_TRIUMF']                       = 15.2266860949
corepower_agis['ANALY_TRIUMF_DOCKER']                = 15.2266860949
corepower_agis['ANALY_TRIUMF_HIMEM']                 = 15.2266860949
corepower_agis['ANALY_TRIUMF_PPS']                   = 15.2266860949
corepower_agis['ANALY_TRIUMF_SL7']                   = 15.2266860949
corepower_agis['ANALY_TRIUMF_TEST']                  = 15.2266860949
corepower_agis['ANALY_UAM']                          = 12.9004576659
corepower_agis['ANALY_UIO']                          = 12.0302510131
corepower_agis['ANALY_UNIBE-LHEP']                   = 10.7213322091
corepower_agis['ANALY_UNIBE-LHEP-UBELIX']            = 10.7213322091
corepower_agis['ANALY_UNLP']                         = 10.00001
corepower_agis['ANALY_UTA_DANILA_TEST']              = 7.54953145917
corepower_agis['ANALY_UTA_PAUL_TEST']                = 7.54953145917
corepower_agis['ANALY_UTFSM']                        = 11.5651041667
corepower_agis['ANALY_VICTORIA']                     = 14.3604166667
corepower_agis['ANALY_VICTORIA_TEST']                = 14.3604166667
corepower_agis['ANALY_WEIZMANN-CREAM']               = 11.75
corepower_agis['ANALY_WISC_ATLAS']                   = 10.00001
corepower_agis['ANALY_wuppertalprod']                = 9.80005411255
corepower_agis['ANALY_ZA-UJ']                        = 10.00001
corepower_agis['ANALY_ZA-WITS-CORE']                 = 10.00001
corepower_agis['ANLASC']                             = 10.00001
corepower_agis['ARC-ES']                             = 12.0302510131
corepower_agis['ARC-TEST']                           = 12.0302510131
corepower_agis['ARGO_ANLASC']                        = 10.0
corepower_agis['ARGO_Cori']                          = 10.0
corepower_agis['ARGO_Edison']                        = 10.0
corepower_agis['ARGO_Mira']                          = 1.0
corepower_agis['Arizona']                            = 0.0
corepower_agis['ARNES']                              = 10.00001
corepower_agis['ARNES_MCORE']                        = 10.00001
corepower_agis['ATLAS_OPP_OSG-CIT_CMS_T2']           = 9.98157248157
corepower_agis['ATLAS_OPP_OSG-CLEMSON_PALMETTO']     = 9.98157248157
corepower_agis['ATLAS_OPP_OSG_ES-PURDUE']            = 9.98157248157
corepower_agis['ATLAS_OPP_OSG_ES-UCONN']             = 9.98157248157
corepower_agis['ATLAS_OPP_OSG-FNAL_GPCE01']          = 9.98157248157
corepower_agis['ATLAS_OPP_OSG-FNAL_GPCE02']          = 9.98157248157
corepower_agis['ATLAS_OPP_OSG-GLOW']                 = 9.98157248157
corepower_agis['ATLAS_OPP_OSG-HYAK']                 = 9.98157248157
corepower_agis['ATLAS_OPP_OSG_MCORE-UCSDT2']         = 9.98157248157
corepower_agis['ATLAS_OPP_OSG-MIT_CMS']              = 9.98157248157
corepower_agis['ATLAS_OPP_OSG-NYSGRID_CORNELL_NYS1'] = 9.98157248157
corepower_agis['ATLAS_OPP_OSG-SU_OG']                = 9.98157248157
corepower_agis['ATLAS_OPP_OSG-USCMS-FNAL-WC1-CE3']   = 9.98157248157
corepower_agis['Australia-ATLAS']                    = 11.375
corepower_agis['Australia-ATLAS_MCORE']              = 11.375
corepower_agis['Australia-ATLAS_MCORE_TEST']         = 11.375
corepower_agis['Australia-ATLAS_MCORE_TEST2']        = 11.375
corepower_agis['Australia-ATLAS_TEST']               = 11.375
corepower_agis['Australia-ATLAS_TEST2']              = 11.375
corepower_agis['BEIJING']                            = 18.5810810811
corepower_agis['BEIJING-CS-TH-1A_MCORE']             = 18.5810810811
corepower_agis['BEIJING-ERAII_MCORE']                = 18.5810810811
corepower_agis['BEIJING-ERA_MCORE']                  = 18.5810810811
corepower_agis['BEIJING_MCORE']                      = 18.5810810811
corepower_agis['BEIJING-PI_MCORE']                   = 18.5810810811
corepower_agis['BEIJING-TIANJIN-TH-1A_MCORE']        = 18.5810810811
corepower_agis['BNL_ATLAS_2']                        = 9.98157248157
corepower_agis['BNL_ATLAS_Install']                  = 9.98157248157
corepower_agis['BNL_CLOUD']                          = 9.98157248157
corepower_agis['BNL_CLOUD_MCORE']                    = 9.98157248157
corepower_agis['BNL_EC2E1']                          = 9.98157248157
corepower_agis['BNL_EC2E1_MCORE']                    = 9.98157248157
corepower_agis['BNL_EC2W1']                          = 9.98157248157
corepower_agis['BNL_EC2W1_MCORE']                    = 9.98157248157
corepower_agis['BNL_EC2W2']                          = 9.98157248157
corepower_agis['BNL_EC2W2_MCORE']                    = 9.98157248157
corepower_agis['BNL_ITB_Install']                    = 9.98157248157
corepower_agis['BNL_ITB_Test1']                      = 9.98157248157
corepower_agis['BNL_LOCAL']                          = 9.98157248157
corepower_agis['BNL_OSG_1']                          = 9.98157248157
corepower_agis['BNL_PROD']                           = 9.98157248157
corepower_agis['BNL_PROD_MCORE']                     = 9.98157248157
corepower_agis['BNL_PROD_MCOREHIMEM']                = 9.98157248157
corepower_agis['BNL_SITE_GK02']                      = 9.98157248157
corepower_agis['BNL_Test_2_CE_1']                    = 9.98157248157
corepower_agis['BOINC']                              = 10.00001
corepower_agis['BOINC_MCORE']                        = 10.00001
corepower_agis['BOINC-TEST']                         = 10.00001
corepower_agis['BU_ATLAS_Tier2_MCORE']               = 20.0126760563
corepower_agis['BU_ATLAS_Tier2_SL6']                 = 20.0126760563
corepower_agis['CA-JADE']                            = 14.5
corepower_agis['CA-MCGILL-CLUMEQ-T2']                = 14.5
corepower_agis['CA-MCGILL-CLUMEQ-T2_DOCKER']         = 14.5
corepower_agis['CA-MCGILL-CLUMEQ-T2_MCORE']          = 14.5
corepower_agis['CA-SCINET-T2']                       = 12.5
corepower_agis['CA-SCINET-T2_MCORE']                 = 12.5
corepower_agis['CA-VICTORIA-WESTGRID-T2']            = 14.3604166667
corepower_agis['CA-VICTORIA-WESTGRID-T2_MCORE']      = 14.3604166667
corepower_agis['CERN-EXTENSION_MCORE']               = 9.72511065833
corepower_agis['CERN-P1']                            = 9.72511065833
corepower_agis['CERN-P1_DYNAMIC_MCORE']              = 9.72511065833
corepower_agis['CERN-P1_DYNAMIC_MCORE_LOWMEM']       = 9.72511065833
corepower_agis['CERN-P1_DYNAMIC_SCORE']              = 9.72511065833
corepower_agis['CERN-P1_MCORE']                      = 9.72511065833
corepower_agis['CERN-P1_MCORE_HI']                   = 9.72511065833
corepower_agis['CERN-P1_preprod']                    = 9.72511065833
corepower_agis['CERN-P1_preprod_MCORE']              = 9.72511065833
corepower_agis['CERN-PROD']                          = 9.72511065833
corepower_agis['CERN-PROD_CLOUD']                    = 9.72511065833
corepower_agis['CERN-PROD_CLOUD_MCORE']              = 9.72511065833
corepower_agis['CERN-PROD_HI']                       = 9.72511065833
corepower_agis['CERN-PROD_MCORE']                    = 9.72511065833
corepower_agis['CERN-PROD-preprod']                  = 9.72511065833
corepower_agis['CERN-PROD-preprod_MCORE']            = 9.72511065833
corepower_agis['CERN-PROD_PRESERVATION']             = 9.72511065833
corepower_agis['CERN-PROD_SHORT']                    = 9.72511065833
corepower_agis['CERN-PROD_T0_4MCORE']                = 9.72511065833
corepower_agis['CERN-PROD_T0_8MCORE']                = 9.72511065833
corepower_agis['CHARMM']                             = 10.00001
corepower_agis['CONNECT']                            = 10.2945472249
corepower_agis['CONNECT_BLUEWATERS']                 = 10.2945472249
corepower_agis['CONNECT_BLUEWATERS_MCORE']           = 10.2945472249
corepower_agis['CONNECT_ES']                         = 10.2945472249
corepower_agis['CONNECT_ES_MCORE']                   = 10.2945472249
corepower_agis['CONNECT_FRESNOSTATE']                = 10.2945472249
corepower_agis['CONNECT_ICCHEPT3']                   = 10.2945472249
corepower_agis['CONNECT_ICCHEPT3_MCORE']             = 10.2945472249
corepower_agis['CONNECT_JETSTREAM']                  = 10.2945472249
corepower_agis['CONNECT_JETSTREAM_MCORE']            = 10.2945472249
corepower_agis['CONNECT_KARST']                      = 10.2945472249
corepower_agis['CONNECT_KARST_MCORE']                = 10.2945472249
corepower_agis['CONNECT_MCORE']                      = 10.2945472249
corepower_agis['CONNECT_MIDWAY']                     = 10.2945472249
corepower_agis['CONNECT_MIDWAY_MCORE']               = 10.2945472249
corepower_agis['CONNECT_ODYSSEY']                    = 10.2945472249
corepower_agis['CONNECT_RODEO']                      = 10.2945472249
corepower_agis['CONNECT_RODEO_MCORE']                = 10.2945472249
corepower_agis['CONNECT_UIUC']                       = 10.2945472249
corepower_agis['CONNECT_UIUC_MCORE']                 = 10.2945472249
corepower_agis['CPPM']                               = 10.879979571
corepower_agis['CPPM_MCORE']                         = 10.879979571
corepower_agis['CSCS-LCG2']                          = 11.5520238713
corepower_agis['CSCS-LCG2-HPC']                      = 11.5520238713
corepower_agis['CSCS-LCG2-HPC_MCORE']                = 11.5520238713
corepower_agis['CSCS-LCG2_MCORE']                    = 11.5520238713
corepower_agis['CYFRONET-LCG2']                      = 14.1999671916
corepower_agis['CYFRONET-LCG2_MCORE']                = 14.1999671916
corepower_agis['DCSC']                               = 12.0302510131
corepower_agis['DCSC_MCORE']                         = 12.0302510131
corepower_agis['DESY-HH']                            = 11.2781627838
corepower_agis['DESY-HH_Condor']                     = 11.2781627838
corepower_agis['DESY-HH_MCORE']                      = 11.2781627838
corepower_agis['DESY-HH_MCORE_Condor']               = 11.2781627838
corepower_agis['DESY-ZN']                            = 16.1600506757
corepower_agis['DESY-ZN_MCORE']                      = 16.1600506757
corepower_agis['EELA-UNLP']                          = 10.00001
corepower_agis['EELA-UTFSM']                         = 11.5651041667
corepower_agis['EELA-UTFSM_MCORE']                   = 11.5651041667
corepower_agis['ES_ORNL_Titan']                      = 9.98157248157
corepower_agis['FIAN_MCORE']                         = 12.1
corepower_agis['FIAN_PROD']                          = 12.1
corepower_agis['Firefly_SBGRID']                     = 126.900773196
corepower_agis['FMPhI-UNIBA']                        = 10.5133928571
corepower_agis['FMPhI-UNIBA_MCORE']                  = 10.5133928571
corepower_agis['FZK-LCG2']                           = 13.0793336979
corepower_agis['FZK-LCG2_HIMEM']                     = 13.0793336979
corepower_agis['FZK-LCG2_MCORE']                     = 13.0793336979
corepower_agis['FZK-LCG2_MCORE_HI']                  = 13.0793336979
corepower_agis['FZK-LCG2_MCORE_LO']                  = 13.0793336979
corepower_agis['FZK-LCG2_TEST']                      = 13.0793336979
corepower_agis['GoeGrid']                            = 10.2658730159
corepower_agis['GoeGrid_MCORE']                      = 10.2658730159
corepower_agis['GOOGLE_COMPUTE_ENGINE']              = 10.00001
corepower_agis['GR-01-AUTH']                         = 10.00001
corepower_agis['GR-12-TEIKAV']                       = 64.0
corepower_agis['GRIDPP_CLOUD']                       = 10.00001
corepower_agis['GRIDPP_MCORE']                       = 10.00001
corepower_agis['GRIF-IRFU']                          = 10.7483226968
corepower_agis['GRIF-IRFU_MCORE']                    = 10.7483226968
corepower_agis['GRIF-LAL_HTCondor']                  = 10.7483226968
corepower_agis['GRIF-LAL_HTCondor_MCORE']            = 10.7483226968
corepower_agis['GRIF-LPNHE']                         = 10.7483226968
corepower_agis['GRIF-LPNHE_MCORE']                   = 10.7483226968
corepower_agis['Harvard-East_SBGRID']                = 9.98157248157
corepower_agis['Harvard-Exp_SBGRID']                 = 9.98157248157
corepower_agis['HELIX_NEBULA_ATOS']                  = 10.00001
corepower_agis['HELIX_NEBULA_EGI']                   = 10.00001
corepower_agis['HEPHY-UIBK']                         = 8.45769230769
corepower_agis['HPC2N']                              = 12.0302510131
corepower_agis['HPC2N_MCORE']                        = 12.0302510131
corepower_agis['HU_ATLAS_Tier2']                     = 12.5894097222
corepower_agis['HU_ATLAS_Tier2_MCORE']               = 12.5894097222
corepower_agis['IAAS']                               = 14.3604166667
corepower_agis['IAAS_MCORE']                         = 14.3604166667
corepower_agis['IAAS_MCORE_HIMEM']                   = 14.3604166667
corepower_agis['IEPSAS-Kosice']                      = 15.0191780822
corepower_agis['IEPSAS-Kosice_MCORE']                = 15.0191780822
corepower_agis['IFAE']                               = 12.1205357143
corepower_agis['IFAE_MCORE']                         = 12.1205357143
corepower_agis['IFIC']                               = 12.9677419355
corepower_agis['IFIC_MCORE']                         = 12.9677419355
corepower_agis['IHEP_MCORE']                         = 8.79993297587
corepower_agis['IHEP_PROD']                          = 8.79993297587
corepower_agis['IL-TAU-HEP']                         = 11.7509363296
corepower_agis['IL-TAU-HEP_MCORE']                   = 11.7509363296
corepower_agis['IN2P3-CC']                           = 10.2642484677
corepower_agis['IN2P3-CC_HIMEM']                     = 10.2642484677
corepower_agis['IN2P3-CC_MCORE']                     = 10.2642484677
corepower_agis['IN2P3-CC_MCORE_HIMEM']               = 10.2642484677
corepower_agis['IN2P3-CC-T3_MCORE']                  = 9.7
corepower_agis['IN2P3-CC-T3_VM01']                   = 12.0
corepower_agis['IN2P3-CC-T3_VM02']                   = 10.0
corepower_agis['IN2P3-CC_VVL']                       = 10.2642484677
corepower_agis['IN2P3-LAPP-TEST']                    = 11.2559633028
corepower_agis['IN2P3-LPSC']                         = 12.0
corepower_agis['IN2P3-LPSC_CLOUD']                   = 12.0
corepower_agis['IN2P3-LPSC_CLOUD_MCORE']             = 12.0
corepower_agis['IN2P3-LPSC_VAC']                     = 12.0
corepower_agis['INFN-BOLOGNA-T3']                    = 10.00001
corepower_agis['INFN-COSENZA']                       = 10.00001
corepower_agis['INFN-COSENZA-RECAS']                 = 8.503
corepower_agis['INFN-COSENZA-RECAS_MCORE']           = 8.503
corepower_agis['INFN-FRASCATI']                      = 9.21590909091
corepower_agis['INFN-FRASCATI_MCORE']                = 9.21590909091
corepower_agis['INFN-GENOVA']                        = 10.00001
corepower_agis['INFN-LECCE']                         = 10.00001
corepower_agis['INFN-MILANO-ATLASC']                 = 8.20586592179
corepower_agis['INFN-MILANO-ATLASC_MCORE']           = 8.20586592179
corepower_agis['INFN-NAPOLI-ATLAS']                  = 8.50280269058
corepower_agis['INFN-NAPOLI-ATLAS_MCORE']            = 8.50280269058
corepower_agis['INFN-NAPOLI-RECAS']                  = 8.50280269058
corepower_agis['INFN-NAPOLI-RECAS_MCORE']            = 8.50280269058
corepower_agis['INFN-NAPOLI-RECAS-TEST']             = 8.50280269058
corepower_agis['INFN-NAPOLI-SCOPE']                  = 8.50280269058
corepower_agis['INFN-ROMA1']                         = 9.33066239316
corepower_agis['INFN-ROMA1_MCORE']                   = 9.33066239316
corepower_agis['INFN-ROMA2']                         = 10.00001
corepower_agis['INFN-ROMA3']                         = 10.00001
corepower_agis['INFN-ROMA3_MCORE']                   = 10.00001
corepower_agis['INFN-T1']                            = 10.91
corepower_agis['INFN-T1-BA']                         = 10.91
corepower_agis['INFN-T1_HIMEM']                      = 10.91
corepower_agis['INFN-T1_MCORE']                      = 10.91
corepower_agis['INFN-T1_MCORE_CP']                   = 10.91
corepower_agis['INFN-T1_TEST']                       = 10.91
corepower_agis['ITEP_MCORE']                         = 13.4051724138
corepower_agis['ITEP_PROD']                          = 13.4051724138
corepower_agis['JINR_MCORE']                         = 12.6984042553
corepower_agis['JINR_PROD']                          = 12.6984042553
corepower_agis['LAPP']                               = 11.2559633028
corepower_agis['LAPP_MCORE']                         = 11.2559633028
corepower_agis['LPC']                                = 12.1723000824
corepower_agis['LPC_MCORE']                          = 12.1723000824
corepower_agis['LPSC_MCORE']                         = 12.0
corepower_agis['LRZ-LMU']                            = 9.69296461976
corepower_agis['LRZ-LMU_C2PAP']                      = 9.69296461976
corepower_agis['LRZ-LMU_C2PAP_ES_MCORE']             = 9.69296461976
corepower_agis['LRZ-LMU_C2PAP_MCORE']                = 9.69296461976
corepower_agis['LRZ-LMU_CLOUD']                      = 9.69296461976
corepower_agis['LRZ-LMU_MCORE']                      = 9.69296461976
corepower_agis['LRZ-LMU_MCORE_LO']                   = 9.69296461976
corepower_agis['LRZ-LMU_MUC1_MCORE']                 = 8.0
corepower_agis['LRZ-LMU_MUC_MCORE1']                 = 8.0
corepower_agis['LRZ-LMU_TEST']                       = 9.69296461976
corepower_agis['Lucille_CE']                         = 11.1164772727
corepower_agis['Lucille_MCORE']                      = 11.1164772727
corepower_agis['LUNARC']                             = 11.2483237548
corepower_agis['LUNARC_MCORE']                       = 11.2483237548
corepower_agis['LUNARC_TEST']                        = 11.2483237548
corepower_agis['MPPMU']                              = 11.3697916667
corepower_agis['MPPMU-HYDRA_MCORE']                  = 11.3697916667
corepower_agis['MPPMU_MCORE']                        = 11.3697916667
corepower_agis['MWT2_HIMEM']                         = 10.2945472249
corepower_agis['MWT2_HIMEM_MCORE']                   = 10.2945472249
corepower_agis['MWT2_MCORE']                         = 10.2945472249
corepower_agis['MWT2_SL6']                           = 10.2945472249
corepower_agis['NCG-INGRID-PT_MCORE']                = 8.01984126984
corepower_agis['NCG-INGRID-PT_SL6']                  = 8.01984126984
corepower_agis['Nebraska_SBGRID']                    = 126.900773196
corepower_agis['NERSC_Cori']                         = 19.91
corepower_agis['NERSC_Edison']                       = 9.98157248157
corepower_agis['NERSC-PDSF']                         = 4.1095890411
corepower_agis['NIKHEF-ELPROD']                      = 13.9899445215
corepower_agis['NIKHEF-ELPROD-HighMem']              = 13.9899445215
corepower_agis['NIKHEF-ELPROD_LONG']                 = 13.9899445215
corepower_agis['NIKHEF-ELPROD_MCORE']                = 13.9899445215
corepower_agis['NSC']                                = 12.0302510131
corepower_agis['NSC_MCORE']                          = 12.0302510131
corepower_agis['ORNL_Titan_MCORE']                   = 5.12
corepower_agis['OUHEP_OSG']                          = 12.921875
corepower_agis['OU_OCHEP_SWT2']                      = 12.921875
corepower_agis['OU_OSCER_ATLAS']                     = 13.8537234043
corepower_agis['OU_OSCER_ATLAS_MCORE']               = 13.8537234043
corepower_agis['OU_OSCER_ATLAS_OPP']                 = 13.8537234043
corepower_agis['pic']                                = 12.1286846276
corepower_agis['pic_HIMEM']                          = 12.1286846276
corepower_agis['pic_HTCondor']                       = 12.1286846276
corepower_agis['pic_IPV6']                           = 12.1286846276
corepower_agis['pic_MCORE']                          = 12.1286846276
corepower_agis['PNPI_PROD']                          = 11.0
corepower_agis['praguelcg2']                         = 8.62183235867
corepower_agis['praguelcg2_MCORE']                   = 8.62183235867
corepower_agis['praguelcg2_TEST']                    = 8.62183235867
corepower_agis['Prairiefire_SBGRID']                 = 126.900773196
corepower_agis['PSNC']                               = 51.9035369775
corepower_agis['RAL-LCG2-ECHO']                      = 10.0
corepower_agis['RAL-LCG2_ES']                        = 10.0
corepower_agis['RAL-LCG2_HIMEM_SL6']                 = 10.0
corepower_agis['RAL-LCG2_MCORE']                     = 10.0
corepower_agis['RAL-LCG2_SL6']                       = 10.0
corepower_agis['RAL-LCG2_VAC']                       = 10.0
corepower_agis['RAL-LCG2_VHIMEM']                    = 10.0
corepower_agis['RDIGTEST']                           = 9.98157248157
corepower_agis['ROMANIA02']                          = 9.6875
corepower_agis['ROMANIA02_MCORE']                    = 9.6875
corepower_agis['ROMANIA07']                          = 10.5637903653
corepower_agis['ROMANIA07_ARC']                      = 10.5637903653
corepower_agis['ROMANIA07_MCORE']                    = 10.5637903653
corepower_agis['ROMANIA14']                          = 9.0
corepower_agis['ROMANIA14_MCORE']                    = 9.0
corepower_agis['ROMANIA16']                          = 9.0
corepower_agis['ROMANIA16_MCORE']                    = 9.0
corepower_agis['RRC-KI-HPC2']                        = 15.5
corepower_agis['RRC-KI_MCORE']                       = 11.9201520913
corepower_agis['RRC-KI_PROD']                        = 11.9201520913
corepower_agis['RRC-KI-T1']                          = 15.5
corepower_agis['RRC-KI-T1_MCORE']                    = 15.5
corepower_agis['RRC-KI-T1_TEST']                     = 15.5
corepower_agis['RRC-KI_TEST_DAN']                    = 11.9201520913
corepower_agis['SARA-MATRIX']                        = 17.2100446429
corepower_agis['SARA-MATRIX_LONG']                   = 17.2100446429
corepower_agis['SARA-MATRIX_MCORE']                  = 17.2100446429
corepower_agis['SFU-LCG2']                           = 14.5
corepower_agis['SFU-LCG2_ES']                        = 14.5
corepower_agis['SFU-LCG2_MCORE']                     = 14.5
corepower_agis['SiGNET']                             = 9.79996886675
corepower_agis['SiGNET_MCORE']                       = 9.79996886675
corepower_agis['SLAC_ES']                            = 12.4958823529
corepower_agis['SLAC_PAUL_TEST']                     = 12.4958823529
corepower_agis['SLACXRD']                            = 12.4958823529
corepower_agis['SLACXRD_LMEM']                       = 12.4958823529
corepower_agis['SLACXRD_MP8']                        = 12.4958823529
corepower_agis['SWT2_CPB']                           = 9.65842245989
corepower_agis['SWT2_CPB_MCORE']                     = 9.65842245989
corepower_agis['Taiwan-LCG2']                        = 10.14
corepower_agis['Taiwan-LCG2_HIMEM']                  = 10.14
corepower_agis['Taiwan-LCG2-HPC']                    = 10.14
corepower_agis['Taiwan-LCG2-HPC_HIMEM']              = 10.14
corepower_agis['Taiwan-LCG2-HPC_MCORE']              = 10.14
corepower_agis['Taiwan-LCG2-HPC_VL']                 = 10.14
corepower_agis['Taiwan-LCG2_MCORE']                  = 10.14
corepower_agis['Taiwan-LCG2_VL']                     = 10.14
corepower_agis['TECHNION-HEP']                       = 11.75
corepower_agis['TECHNION-HEP_MCORE']                 = 11.75
corepower_agis['TESTGLEXEC']                         = 9.98157248157
corepower_agis['TestPilot']                          = 9.98157248157
corepower_agis['TOKYO']                              = 18.1097005208
corepower_agis['TOKYO_ARC']                          = 18.1097005208
corepower_agis['TOKYO_MCORE']                        = 18.1097005208
corepower_agis['TOKYO_MCORE_ARC']                    = 18.1097005208
corepower_agis['TR-10-ULAKBIM']                      = 5.19097222222
corepower_agis['TR-10-ULAKBIM_MCORE']                = 5.19097222222
corepower_agis['TRIUMF']                             = 15.2266860949
corepower_agis['TRIUMF_DOCKER']                      = 15.2266860949
corepower_agis['TRIUMF_DOCKER_MCORE']                = 15.2266860949
corepower_agis['TRIUMF_HIMEM']                       = 15.2266860949
corepower_agis['TRIUMF_MCORE']                       = 15.2266860949
corepower_agis['TRIUMF_MCORE_LOMEM']                 = 15.2266860949
corepower_agis['TRIUMF_PPS']                         = 15.2266860949
corepower_agis['TRIUMF_SL7']                         = 15.2266860949
corepower_agis['TRIUMF_SL7_MCORE']                   = 15.2266860949
corepower_agis['TRIUMF_TEST']                        = 15.2266860949
corepower_agis['TUDresden-ZIH']                      = 10.00001
corepower_agis['TW-FTT_MCORE']                       = 10.1397058824
corepower_agis['TW-FTT_SL6']                         = 10.1397058824
corepower_agis['TWTEST']                             = 9.98157248157
corepower_agis['UAM-LCG2']                           = 12.9004576659
corepower_agis['UAM-LCG2_MCORE']                     = 12.9004576659
corepower_agis['UFlorida-HPC_SBGRID']                = 16.5094339623
corepower_agis['UIO']                                = 12.0302510131
corepower_agis['UIO_MCORE']                          = 12.0302510131
corepower_agis['UIO_MCORE_LOPRI']                    = 12.0302510131
corepower_agis['UKI-ARCHER_MCORE']                   = 12.8996212121
corepower_agis['UKI-LT2-Brunel_MCORE']               = 11.95
corepower_agis['UKI-LT2-Brunel_SL6']                 = 11.95
corepower_agis['UKI-LT2-IC-HEP_MCORE']               = 9.23946188341
corepower_agis['UKI-LT2-IC-HEP_SL6']                 = 9.23946188341
corepower_agis['UKI-LT2-QMUL_HIMEM_SL6']             = 11.2843283582
corepower_agis['UKI-LT2-QMUL_MCORE']                 = 11.2843283582
corepower_agis['UKI-LT2-QMUL_SL6']                   = 11.2843283582
corepower_agis['UKI-LT2-QMUL_TEST']                  = 11.2843283582
corepower_agis['UKI-LT2-RHUL_MCORE']                 = 10.0268229167
corepower_agis['UKI-LT2-RHUL_SL6']                   = 10.0268229167
corepower_agis['UKI-LT2-UCL-HEP_VAC']                = 10.00001
corepower_agis['UKI-NORTHGRID-LANCS-HEP_CLOUD']      = 14.4
corepower_agis['UKI-NORTHGRID-LANCS-HEP_MCORE']      = 14.4
corepower_agis['UKI-NORTHGRID-LANCS-HEP_SL6']        = 14.4
corepower_agis['UKI-NORTHGRID-LIV-HEP_MCORE']        = 10.1992481203
corepower_agis['UKI-NORTHGRID-LIV-HEP_SL6']          = 10.1992481203
corepower_agis['UKI-NORTHGRID-LIV-HEP_VAC']          = 10.1992481203
corepower_agis['UKI-NORTHGRID-MAN-HEP_MCORE']        = 9.85640681004
corepower_agis['UKI-NORTHGRID-MAN-HEP_SL6']          = 9.85640681004
corepower_agis['UKI-NORTHGRID-MAN-HEP_VAC']          = 9.85640681004
corepower_agis['UKI-NORTHGRID-SHEF-HEP_MCORE']       = 13.2
corepower_agis['UKI-NORTHGRID-SHEF-HEP_SL6']         = 13.2
corepower_agis['UKI-SCOTGRID-DURHAM-ARC']            = 13.4003152088
corepower_agis['UKI-SCOTGRID-DURHAM_MCORE']          = 13.4003152088
corepower_agis['UKI-SCOTGRID-DURHAM_SL6']            = 13.4003152088
corepower_agis['UKI-SCOTGRID-ECDF_MCORE']            = 12.8996212121
corepower_agis['UKI-SCOTGRID-ECDF_MCORE_SL7']        = 12.8996212121
corepower_agis['UKI-SCOTGRID-ECDF_SL6']              = 12.8996212121
corepower_agis['UKI-SCOTGRID-ECDF_SL7']              = 12.8996212121
corepower_agis['UKI-SCOTGRID-ECDF_TEST']             = 12.8996212121
corepower_agis['UKI-SCOTGRID-GLASGOW_MCORE']         = 8.740063593
corepower_agis['UKI-SCOTGRID-GLASGOW_SL6']           = 8.740063593
corepower_agis['UKI-SCOTGRID-GLASGOW_TEST']          = 8.740063593
corepower_agis['UKI-SOUTHGRID-BHAM-HEP_SL6']         = 14.1622474747
corepower_agis['UKI-SOUTHGRID-CAM-HEP_MCORE']        = 14.5588235294
corepower_agis['UKI-SOUTHGRID-CAM-HEP_SL6']          = 14.5588235294
corepower_agis['UKI-SOUTHGRID-OX-HEP_CLOUD']         = 10.1547987616
corepower_agis['UKI-SOUTHGRID-OX-HEP_MCORE']         = 10.1547987616
corepower_agis['UKI-SOUTHGRID-OX-HEP_SL6']           = 10.1547987616
corepower_agis['UKI-SOUTHGRID-OX-HEP_TEST']          = 10.1547987616
corepower_agis['UKI-SOUTHGRID-OX-HEP_VAC']           = 10.1547987616
corepower_agis['UKI-SOUTHGRID-RALPP_MCORE']          = 10.0
corepower_agis['UKI-SOUTHGRID-RALPP_SL6']            = 10.0
corepower_agis['UKI-SOUTHGRID-SUSX_MCORE']           = 10.4479166667
corepower_agis['UKI-SOUTHGRID-SUSX_SL6']             = 10.4479166667
corepower_agis['UMESHTEST']                          = 9.98157248157
corepower_agis['UNIBE-LHEP']                         = 10.7213322091
corepower_agis['UNIBE-LHEP_CLOUD']                   = 10.7213322091
corepower_agis['UNIBE-LHEP_CLOUD_MCORE']             = 10.7213322091
corepower_agis['UNIBE-LHEP_MCORE']                   = 10.7213322091
corepower_agis['UNIBE-LHEP-UBELIX']                  = 10.7213322091
corepower_agis['UNIBE-LHEP-UBELIX_MCORE']            = 10.7213322091
corepower_agis['UNI-BONN']                           = 10.00001
corepower_agis['UNI-DORTMUND']                       = 10.00001
corepower_agis['UNI-FREIBURG']                       = 8.42007926024
corepower_agis['UNI-FREIBURG_MCORE']                 = 8.42007926024
corepower_agis['UNIGE-DPNC']                         = 10.00001
corepower_agis['UNIGE-DPNC_MCORE']                   = 10.00001
corepower_agis['UNI-SIEGEN-HEP']                     = 10.00001
corepower_agis['UTA_DANILA_TEST']                    = 7.54953145917
corepower_agis['UTA_PAUL_TEST']                      = 7.54953145917
corepower_agis['UTA_SWT2']                           = 7.54953145917
corepower_agis['UTA_SWT2_Install']                   = 7.54953145917
corepower_agis['UTA_SWT2_MCORE']                     = 7.54953145917
corepower_agis['UW_VDT_ITB']                         = 9.98157248157
corepower_agis['WEIZMANN-LCG2']                      = 11.75
corepower_agis['WEIZMANN-LCG2_MCORE']                = 11.75
corepower_agis['WISC_ATLAS_ES']                      = 10.00001
corepower_agis['WQCG-Harvard-OSG_SBGRID']            = 9.98157248157
corepower_agis['WT2_Install']                        = 12.4958823529
corepower_agis['wuppertalprod']                      = 9.80005411255
corepower_agis['wuppertalprod_HI']                   = 9.80005411255
corepower_agis['wuppertalprod_MCORE']                = 9.80005411255
corepower_agis['ZA-CHPC']                            = 0.0
corepower_agis['ZA-UJ']                              = 10.00001
corepower_agis['ZA-WITS-CORE']                       = 10.00001

boincsites = ('ANALY_BOINC',
              'BOINC',
              'BOINC-ES',
              'BOINC-ES_CLONE',
              'BOINC_CHECKPOINT',
              'BOINC_MCORE',
              'BOINC_VIRTUAL')

cloudsites = ('ANALY_IAAS',
              'ANALY_IAAS_TEST',
              'CERN-EXTENSION_CERN',
              'CERN-EXTENSION_DE',
              'CERN-EXTENSION_HARVESTER',
              'CERN-EXTENSION_TEST',
              'CERN-EXTENSION_UK',
              'CERN-P1',
              'CERN-P1_DYNAMIC_MCORE',
              'CERN-P1_DYNAMIC_MCORE_LOWMEM',
              'CERN-P1_DYNAMIC_SCORE',
              'HELIX_NEBULA_ATOS',
              'HELIX_NEBULA_EGI',
              'IAAS',
              'IAAS_MCORE',
              'IAAS_MCORE_HIMEM',
              'IN2P3-CC-T3_VM01',
              'IN2P3-CC-T3_VM01_SL7',
              'IN2P3-CC-T3_VM02',
              'IN2P3-CC-T3_VM02_SL7',
              'RAL-AZURE_VAC',
              'RAL-LCG2_VAC',
              'UIO_CLOUD',
              'UKI-LT2-UCL-HEP_VAC',
              'UKI-NORTHGRID-MAN-HEP_MCOREVAC',
              'UKI-SCOTGRID-GLASGOW_MCOREVAC',
              'UKI-SOUTHGRID-BHAM-HEP_MCOREVAC',
              'UKI-SOUTHGRID-OX-HEP_CLOUD',
              'UNIBE-LHEP_CLOUD',
              'UNIBE-LHEP_CLOUD_MCORE')

hpcsites = ('ALCF_Cooley',
            'ALCF_Theta',
            'ALCF_Theta_ES',
            'ANALY_CSCS-HPC',
            'ANALY_HPC2N',
            'ANALY_NSC',
            'ANALY_ORNL_Titan',
            'ANALY_RRC-KI-HPC',
            'ANALY_UIO',
            'Anselm_MCORE',
            'BEIJING-CS-TH-1A_MCORE',
            'BEIJING-ERAII_MCORE',
            'BEIJING-ERA_MCORE',
            'BEIJING-PI_MCORE',
            'BEIJING-TIANJIN-TH-1A_MCORE',
            'BNL_KNL_MCORE',
            'CSCS-LCG2-HPC',
            'CSCS-LCG2-HPC_MCORE',
            'CSCS-LCG2-HPC_MCORE_TEST',
            'CSCS-TODI',
            'ES_ORNL_Titan',
            'HPC2N',
            'HPC2N_MCORE',
            'IN2P3-CC_HPC_DEBUG',
            'IN2P3-CC_HPC_IDRIS_MCORE',
            'IN2P3-CC_HPC_IDRIS_MCORE1',
            'IN2P3-CC_HPC_VIRTUAL',
            'LRZ-LMU_C2PAP',
            'LRZ-LMU_C2PAP_MCORE',
            'LRZ-LMU_MUC1_MCORE',
            'LRZ-LMU_MUC_MCORE',
            'LRZ-LMU_MUC_MCORE1',
            'MPPMU-DRACO_MCORE',
            'MPPMU-HYDRA_MCORE',
            'NERSC_Cori',
            'NERSC_Cori_2',
            'NERSC_Cori_p1_ES',
            'NERSC_Cori_p1_mcore',
            'NERSC_Cori_p2_ES',
            'NERSC_Cori_p2_mcore',
            'NERSC_Edison',
            'NERSC_Edison_ES',
            'NERSC_Edison_mcore',
            'NSC',
            'NSC_MCORE',
            'ORNL_Titan_MCORE',
            'ORNL_Titan_VIRTUAL',
            'RRC-KI-HPC2',
            'Titan_Harvester_MCORE',
            'Titan_Harvester_test_MCORE',
            'Titan_long_MCORE',
            'UIO',
            'UIO_ARC_TEST',
            'UIO_MCORE',
            'UIO_MCORE_LOPRI')

### select period

In [2]:
start_date = datetime.datetime(2017, 11, 1, 0, 0, 0)
#start_date = datetime.datetime(2018, 1, 30, 0, 0, 0)
curre_date = datetime.datetime(2018, 1, 31, 23, 59, 59)
#curre_date = datetime.datetime.utcnow()
days_of_data = (curre_date - start_date).days
timemargin = datetime.timedelta(1)

### selecting indices to scan

In [3]:
# get job archive indices
indices = es.cat.indices(index="jobs_archive_20*", h="index", request_timeout=600).split('\n')
indices = sorted(indices)
indices = [x for x in indices if x != '']
#for i in indices:
#    print(i)

selected_indices=[] 
for d in range(days_of_data+1):
    td = start_date + datetime.timedelta(d)
    ind = 'jobs_archive_'+td.strftime('%Y-%m-%d')
    if ind in indices:
        selected_indices.append(ind)

job_indices = ','.join(selected_indices)
print(job_indices)

jobs_archive_2017-11-01,jobs_archive_2017-11-02,jobs_archive_2017-11-03,jobs_archive_2017-11-04,jobs_archive_2017-11-05,jobs_archive_2017-11-06,jobs_archive_2017-11-07,jobs_archive_2017-11-08,jobs_archive_2017-11-09,jobs_archive_2017-11-10,jobs_archive_2017-11-11,jobs_archive_2017-11-12,jobs_archive_2017-11-13,jobs_archive_2017-11-14,jobs_archive_2017-11-15,jobs_archive_2017-11-16,jobs_archive_2017-11-17,jobs_archive_2017-11-18,jobs_archive_2017-11-19,jobs_archive_2017-11-20,jobs_archive_2017-11-21,jobs_archive_2017-11-22,jobs_archive_2017-11-23,jobs_archive_2017-11-24,jobs_archive_2017-11-25,jobs_archive_2017-11-26,jobs_archive_2017-11-27,jobs_archive_2017-11-28,jobs_archive_2017-11-29,jobs_archive_2017-11-30,jobs_archive_2017-12-01,jobs_archive_2017-12-02,jobs_archive_2017-12-03,jobs_archive_2017-12-04,jobs_archive_2017-12-05,jobs_archive_2017-12-06,jobs_archive_2017-12-07,jobs_archive_2017-12-08,jobs_archive_2017-12-09,jobs_archive_2017-12-10,jobs_archive_2017-12-11,jobs_archive_201

### running the scan

In [4]:
job_query = {
    "size": 0,
    "_source": [ "pandaid", "corecount", "cloud", "jobstatus", "processingtype", "actualcorecount",
                 "nevents", "cpuconsumptiontime",
                 "maxwalltime", "timeExe", "wall_time",
                 "hs06", "hs06sec", "computingsite", "cpu_eff", "creationhost"],
    'query':{
#        "match_all": {}
             'bool':{
                    'must':[
                         { "term": {"jobstatus": "finished" } },
#                         { "regexp": {"creationhost" : "hammercloud-ai*" } },
                         { "term": {"nevents" : "25" } },
#                         { "term": {"prodsourcelabel": "managed" } }
                     ],
#                    'should':[
                       # { "term": {"prodsourcelabel": "rc_test" } },
                       # { "term": {"prodsourcelabel": "prod_test" } },
                       # { "term": {"prodsourcelabel": "managed" } }
#                        { "term": {"jobstatus": "finished" } },
#                        { "term": {"jobstatus": "failed" } },
#                    ]
             }
    }
                
}

scroll = scan(client=es, index=job_indices, query=job_query, scroll='1000000m', timeout="1000000m", size=10000)

count = 0
for res in scroll:
    count += 1
    
    #print(res)
    #if count>10000: break
    
    r = res['_source']
    pandaid = r['pandaid']
    cores = r['corecount']
    cloud = r['cloud']
    jobstatus = r['jobstatus']
    processingtype = r['processingtype']
    actualcorecount = r['actualcorecount']
    nevents  = r['nevents']
    cpuconsumptiontime  = r['cpuconsumptiontime']
    maxwalltime  = r['maxwalltime']
    timeExe  = r['timeExe']
    wall_time  = r['wall_time']
    hs06  = r['hs06']
    hs06sec  = r['hs06sec']
    computingsite  = r['computingsite']
    cpu_eff = r['cpu_eff']
    creationhost = r['creationhost']
    
    if creationhost is None:
        continue
        
#    if "hammercloud-ai" not in creationhost:
#        continue
        
#    if nevents != 25:
#        continue
        
    if wall_time == 0:
        continue
            
    if actualcorecount != None and actualcorecount != 0:
        if hs06 != None:
            hs06      /= actualcorecount
        wall_time *= actualcorecount
    elif cores != None and cores != 0:
        if hs06 != None:
            hs06      /= cores
        wall_time *= cores

    # use corepower in AGIS
    if computingsite in corepower_agis:
        hs06 = corepower_agis[computingsite]
    else :
        hs06 = 0

    # convert to cpu_time
    #wall_time *= cpu_eff
    
    if not count%100000: 
        print(count)
        #print(data)
    
    resource_type = 'grid'
    if computingsite in boincsites:
        resource_type = 'boinc'
    elif computingsite in cloudsites:
        resource_type = 'cloud'
    elif computingsite in hpcsites:
        resource_type = 'hpc'
        
    if resource_type == 'grid':
        if actualcorecount == 1:
            resource_type = 'grid1'
        elif actualcorecount == 8:
            resource_type = 'grid8'
    
    jobs.append((pandaid, cores, cloud, jobstatus, processingtype, actualcorecount,
                 nevents, wall_time, hs06, hs06sec, resource_type, cpu_eff))
    if count < 100:
        print(pandaid)
        print(jobs[-1])

#job=pd.DataFrame(data,columns=['initialjob', 'cores', 'errorcode', 'pandaid', 'failedattempt', 'child_ids', 'wall_time', 'jobstatus', 'processingtype'])
#del data
print('jobs = '+str(len(jobs)))

3732348564
(3732348564, 1, 'FR', 'finished', 'gangarobot-celt', 1, 25, 9014, 10.7483226968, None, 'grid1', 0.657089)
3762010349
(3762010349, 1, 'DE', 'finished', 'gangarobot-celt', 1, 25, 4181, 8.62183235867, None, 'grid1', 0.95886153)
3801770274
(3801770274, 1, 'CA', 'finished', 'gangarobot-celt', 1, 25, 8795, 14.3604166667, None, 'cloud', 0.95054007)
3712282416
(3712282416, 1, 'DE', 'finished', 'gangarobot-celt', 1, 25, 5838, 9.80005411255, None, 'grid1', 0.9616307)
3772205406
(3772205406, 1, 'IT', 'finished', 'gangarobot-celt', 1, 25, 7029, 10.00001, None, 'grid1', 0.96400625)
3720819397
(3720819397, 1, 'US', 'finished', 'gangarobot-celt', 1, 25, 6298, 7.54953145917, None, 'grid1', 0.9647507)
3700078752
(3700078752, 1, 'FR', 'finished', 'gangarobot-celt', 1, 25, 5353, 11.2559633028, None, 'grid1', 0.9557258)
3763006853
(3763006853, 1, 'DE', 'finished', 'gangarobot-celt', 1, 25, 4733, 10.2658730159, None, 'grid1', 0.9687302)
3694016606
(3694016606, 1, 'UK', 'finished', 'gangarobot-ce

### Print result

In [5]:
count = 0
types   = ('evgen', 'gangarobot-celt', 'simul', 'pile', 'deriv', 'merge', 'recon', 'pmerge', 'eventindex', 'skim', 'overlay')
#ymaxval = (      1,              0.1,    0.1,    0.2,      15,     100,     0.5,       50,            1,   0.01,      0.02)
ymaxval = (  0.005,               0.1,    0.1,    0.1,      10,      10,     0.5,       50,            1,   0.01,      0.02)
yminval = (      0,                 0,      0,      0,       0,       0,       0,        0,            0,      0,         0)

resources = ('grid1', 'grid8', 'cloud', 'hpc')
data = [({}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}),
        ({}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}),
        ({}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}),
        ({}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {})]
minval = [[100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000],
          [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000],
          [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000],
          [100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]]
maxval = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
nval = (1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000)
minhs06 = -0.5
maxhs06 = 50.5
nhs06   = 51

print('start')

for job in jobs:
    cores = job[1]
    cloud = job[2]
    jobstatus = job[3]
    processingtype = job[4]
    actualcorecount = job[5]
    nevents = job[6]
    wall_time = job[7]
    hs06 = job[8]
    resource_type = job[10]

    count += 1
    
    index_res = -1
    for res in resources:
        index_res += 1
        if resource_type == res:
            index = -1
            for type in types:
                index += 1
                if count < 100:
                    print('CHECK: '+processingtype)
                if wall_time == 0:
                    continue
                if processingtype == type:
                    if count < 100:
                        print('PASS:  '+processingtype)
                    if minval[index_res][index] > nevents / wall_time:
                        minval[index_res][index] = nevents / wall_time
                    if maxval[index_res][index] < nevents / wall_time:
                        maxval[index_res][index] = nevents / wall_time
                        
index_res = -1
for res in resources:
    index_res += 1
    index = -1
    for type in types:
        index += 1
        if maxval[index_res][index] > ymaxval[index]:
            maxval[index_res][index] = ymaxval[index]
        if minval[index_res][index] < yminval[index]:
            minval[index_res][index] = yminval[index]
        if maxval[index_res][index] * 1.01 <= minval[index_res][index]:
            maxval[index_res][index] = minval[index_res][index] + 1
               
for job in jobs:
    cores = job[1]
    cloud = job[2]
    jobstatus = job[3]
    processingtype = job[4]
    actualcorecount = job[5]
    nevents = job[6]
    wall_time = job[7]
    hs06 = job[8]
    resource_type = job[10]

    count += 1
    index_res = -1
    for res in resources:
        index_res += 1
        if resource_type == res:
            index = -1
            if wall_time == 0:
                continue
            # avoid strange entries (different hs06 from AGIS) in simul
            # if cores != actualcorecount:
            #    continue
            for type in types:
                index += 1
                if processingtype == type:
                    #print(str(maxval[index_res][index])+' '+str(minval[index_res][index])+' '+str(index_res)+' '+str(index))
                    xindex = int((hs06                - minhs06                 ) / (float(maxhs06                  - minhs06                 ) / nhs06      ))
                    yindex = int((nevents / wall_time - minval[index_res][index]) / (float(maxval[index_res][index] - minval[index_res][index]) / nval[index]))
                    if yindex >= nval[index]:
                        yindex = nval[index] - 1
                    if yindex < 0:
                        yindex = 0
                    if (xindex, yindex) not in data[index_res][index]:
                        data[index_res][index][(xindex, yindex)] = 0
                    data[index_res][index][(xindex, yindex)] += 1

index_res = -1
for res in resources:
    index_res += 1
    index = -1
    for type in types:
        index += 1
        print('=== '+res+' '+type+' ===')
        print(str(nhs06)+' '+str(minhs06)+' '+str(maxhs06))
        print(str(nval[index])+' '+str(minval[index_res][index])+' '+str(maxval[index_res][index]))
        print(data[index_res][index])

start
CHECK: gangarobot-celt
CHECK: gangarobot-celt
PASS:  gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
PASS:  gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
PASS:  gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
PASS:  gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK

CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
PASS:  gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
PASS:  gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
PASS:  gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
CHECK: gangarobot-celt
PASS:  gangarobot-celt
CHECK: gang